In [2]:
# source:https://github.com/ismorphism/DeepECG
# 2019/11/25    YANG Jie    小修改
from sklearn.metrics import confusion_matrix, accuracy_score
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler, RobustScaler
import pandas as pd
import scipy.io as sio
from os import listdir
from os.path import isfile, join
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras import regularizers
from keras.utils import np_utils

number_of_classes = 4  # Total number of classes


def to_one_hot(y):
    return np_utils.to_categorical(y)


def change(x):  # From boolean arrays to decimal arrays
    answer = np.zeros((np.shape(x)[0]))
    for i in range(np.shape(x)[0]):
        max_value = max(x[i, :])
        max_index = list(x[i, :]).index(max_value)
        answer[i] = max_index
    return answer.astype(np.int)


mypath = 'D:/pythonWS/DeepECG-master/training2017/'  # Training directory
onlyfiles = [f for f in listdir(mypath) if (isfile(join(mypath, f)) and f[0] == 'A')]
bats = [f for f in onlyfiles if f[7] == 'm']
check = 100
mats = [f for f in bats if (np.shape(sio.loadmat(mypath + f)['val'])[1] >= check)]
size = len(mats)
print('Total training size is ', size)
big = 10100
X = np.zeros((size, big))

for i in range(size):
    dummy = sio.loadmat(mypath + mats[i])['val'][0, :]
    if (big - len(dummy)) <= 0:
        X[i, :] = dummy[0:big]
    else:
        b = dummy[0:(big - len(dummy))]
        goal = np.hstack((dummy, b))
        while len(goal) != big:
            b = dummy[0:(big - len(goal))]
            goal = np.hstack((goal, b))
        X[i, :] = goal

target_train = np.zeros((size, 1))
Train_data = pd.read_csv(mypath + 'REFERENCE.csv', sep=',', header=None, names=None)
for i in range(size):
    if Train_data.loc[Train_data[0] == mats[i][:6], 1].values == 'N':
        target_train[i] = 0
    elif Train_data.loc[Train_data[0] == mats[i][:6], 1].values == 'A':
        target_train[i] = 1
    elif Train_data.loc[Train_data[0] == mats[i][:6], 1].values == 'O':
        target_train[i] = 2
    else:
        target_train[i] = 3

Label_set = to_one_hot(target_train)

X = (X - X.mean()) / (X.std())  # Some normalization here
X = np.expand_dims(X, axis=2)  # For Keras's data input size

values = [i for i in range(size)]
permutations = np.random.permutation(values)
X = X[permutations, :]
Label_set = Label_set[permutations, :]

train = 0.9  # Size of training set in percentage
X_train = X[:int(train * size), :]
Y_train = Label_set[:int(train * size), :]
X_val = X[int(train * size):, :]
Y_val = Label_set[int(train * size):, :]



Total training size is  8528


In [3]:
# def create_model():
model = Sequential()
model.add(Conv1D(128, 55, activation='relu', input_shape=(big, 1)))
model.add(MaxPooling1D(10))
model.add(Dropout(0.5))
model.add(Conv1D(128, 25, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Dropout(0.5))
model.add(Conv1D(128, 10, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Dropout(0.5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalAveragePooling1D())
# model.add(Flatten())
model.add(Dense(256, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(number_of_classes, kernel_initializer='normal', activation='softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
hist = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), batch_size=256, epochs=50, verbose=2, shuffle=True)
pd.DataFrame(hist.history).to_csv(path_or_buf='Conv_models/History.csv')
predictions = model.predict(X_val)
score = accuracy_score(change(Y_val), change(predictions))
print('Last epoch\'s validation score is ', score)
df = pd.DataFrame(change(predictions))
df.to_csv(path_or_buf='Conv_models/Preds_' + str(format(score, '.4f')) + '.csv', index=None, header=None)
pd.DataFrame(confusion_matrix(change(Y_val), change(predictions))).to_csv(
    path_or_buf='Conv_models/Result_Conf' + str(format(score, '.4f')) + '.csv', index=None, header=None)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 10046, 128)        7168      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1004, 128)        0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 1004, 128)         0         
                                                                 
 conv1d_1 (Conv1D)           (None, 980, 128)          409728    
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 196, 128)         0         
 1D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 196, 128)          0

Epoch 28/50
30/30 - 195s - loss: 0.4183 - accuracy: 0.8491 - val_loss: 0.4641 - val_accuracy: 0.8499 - 195s/epoch - 6s/step
Epoch 29/50
30/30 - 177s - loss: 0.4252 - accuracy: 0.8490 - val_loss: 0.4518 - val_accuracy: 0.8406 - 177s/epoch - 6s/step
Epoch 30/50
30/30 - 180s - loss: 0.4177 - accuracy: 0.8479 - val_loss: 0.4725 - val_accuracy: 0.8417 - 180s/epoch - 6s/step
Epoch 31/50
30/30 - 181s - loss: 0.4186 - accuracy: 0.8455 - val_loss: 0.4465 - val_accuracy: 0.8558 - 181s/epoch - 6s/step
Epoch 32/50
30/30 - 178s - loss: 0.4119 - accuracy: 0.8517 - val_loss: 0.4438 - val_accuracy: 0.8593 - 178s/epoch - 6s/step
Epoch 33/50
30/30 - 183s - loss: 0.4040 - accuracy: 0.8511 - val_loss: 0.4562 - val_accuracy: 0.8558 - 183s/epoch - 6s/step
Epoch 34/50
30/30 - 181s - loss: 0.3994 - accuracy: 0.8556 - val_loss: 0.4299 - val_accuracy: 0.8581 - 181s/epoch - 6s/step
Epoch 35/50
30/30 - 180s - loss: 0.3939 - accuracy: 0.8565 - val_loss: 0.4255 - val_accuracy: 0.8617 - 180s/epoch - 6s/step
Epoch 36

FileNotFoundError: [Errno 2] No such file or directory: 'Conv_models/History.csv'